# HW-3 (<a href="https://www.kaggle.com/c/sfmlo-7-classification">Kaggle task</a>)
#### Команда №3
#### Состав команды:

    Kyrylo Prykhno         as Kyrylo Prykhno
    Александр              as Alexander
    Алексей Совит          as A.Sovit
    Александра Гринцвайг   as ALEXANDRA GRINTSVAYG
    Марат Нуртас	
    Александр Аверичев	
    Александр Святкин      as Alexander Svyatkin
    Евгений Гаврилов       as un1qstyle
    Виктор Верниковский	

In [1]:
# подключаем библиотеки

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
from IPython.display import Image

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (18,12)

### Данные задачи

In [232]:
# читаем данные для обучения и для теста
#
df_train = pd.read_csv('data/train.csv')

df_test1 = pd.read_csv('data/test.csv')
df_test_y = pd.read_csv('data/sample_submission.csv')

df_test = df_test1.merge(df_test_y, on='_id', how='left')



In [233]:
# посмотрим учебные данные
df_train.shape

(24712, 22)

In [234]:
df_train.head()

,_id,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,target
0,df7489733b004bbe40d3d37b34f82419,54,technician,married,professional.course,unknown,no,no,cellular,aug,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,905a0b9a5456ee962223033473666be3,36,services,single,basic.4y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,02d69c07ec6d68e31d641edd45ce61cd,53,blue-collar,married,basic.4y,no,yes,no,cellular,aug,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,0
3,e492fbe0cccaf67cdb5c0944dbc8a167,34,blue-collar,married,basic.9y,no,yes,yes,telephone,may,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,ac2dc6f5835bdea8e71dd99b65208d07,29,services,single,university.degree,no,yes,no,telephone,aug,...,1,999,0,nonexistent,-1.7,94.027,-38.3,0.890,4991.6,0


In [235]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24712 entries, 0 to 24711
Data columns (total 22 columns):
_id               24712 non-null object
age               24712 non-null int64
job               24712 non-null object
marital           24712 non-null object
education         24712 non-null object
default           24712 non-null object
housing           24712 non-null object
loan              24712 non-null object
contact           24712 non-null object
month             24712 non-null object
day_of_week       24712 non-null object
duration          24712 non-null int64
campaign          24712 non-null int64
pdays             24712 non-null int64
previous          24712 non-null int64
poutcome          24712 non-null object
emp.var.rate      24712 non-null float64
cons.price.idx    24712 non-null float64
cons.conf.idx     24712 non-null float64
euribor3m         24712 non-null float64
nr.employed       24712 non-null float64
target            24712 non-null int64
dtypes: floa

### Подготовка данных

In [236]:
def preproc(df_init):
    df_preproc = df_init.copy()
    
    # Удалили признаки
    df_preproc = df_preproc.drop(['_id'], axis=1)
    
    return df_preproc

In [237]:
df_preproc = df_train.pipe(preproc)

df_test_preproc = df_test.pipe(preproc)


for i in df_preproc.columns: # перебираем все столбцы
    if str(df_preproc[i].dtype) == 'object': # если тип столбца - object
        print('='*10)
        print(i) # выводим название столбца
        print(set(df_preproc[i])) # выводим все его значения (но делаем set - чтоб значения не повторялись)
        print('\n') # выводим пустую строку

job
{'technician', 'student', 'management', 'entrepreneur', 'unknown', 'admin.', 'housemaid', 'self-employed', 'retired', 'blue-collar', 'unemployed', 'services'}


marital
{'single', 'married', 'divorced', 'unknown'}


education
{'basic.6y', 'unknown', 'university.degree', 'basic.4y', 'professional.course', 'basic.9y', 'illiterate', 'high.school'}


default
{'no', 'yes', 'unknown'}


housing
{'no', 'yes', 'unknown'}


loan
{'no', 'yes', 'unknown'}


contact
{'telephone', 'cellular'}


month
{'jun', 'oct', 'jul', 'aug', 'nov', 'may', 'apr', 'mar', 'sep', 'dec'}


day_of_week
{'wed', 'mon', 'thu', 'tue', 'fri'}


poutcome
{'nonexistent', 'failure', 'success'}




In [238]:
df_test_preproc.head().T


,0,1,2,3,4
age,57,55,33,36,27
job,technician,unknown,blue-collar,admin.,housemaid
marital,married,married,married,married,married
education,high.school,unknown,basic.9y,high.school,high.school
default,no,unknown,no,no,no
housing,no,yes,no,no,yes
loan,yes,no,no,no,no
contact,cellular,telephone,cellular,telephone,cellular
month,may,may,may,jun,jul
day_of_week,mon,thu,fri,fri,fri


In [239]:
# выделяем из учебных данных вектор целей - target
y = df_preproc['target'].values
y_test = df_test_preproc['target'].values

# убираем из учебных данных колонку target
df_X = df_preproc.drop('target', axis=1)
df_X_test = df_test_preproc.drop('target', axis=1)


df_X.info(verbose=True, memory_usage=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24712 entries, 0 to 24711
Data columns (total 20 columns):
age               24712 non-null int64
job               24712 non-null object
marital           24712 non-null object
education         24712 non-null object
default           24712 non-null object
housing           24712 non-null object
loan              24712 non-null object
contact           24712 non-null object
month             24712 non-null object
day_of_week       24712 non-null object
duration          24712 non-null int64
campaign          24712 non-null int64
pdays             24712 non-null int64
previous          24712 non-null int64
poutcome          24712 non-null object
emp.var.rate      24712 non-null float64
cons.price.idx    24712 non-null float64
cons.conf.idx     24712 non-null float64
euribor3m         24712 non-null float64
nr.employed       24712 non-null float64
dtypes: float64(5), int64(5), object(10)
memory usage: 3.8+ MB


### Библиотеки

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import roc_auc_score

In [11]:
from scipy.stats import randint as randint
from scipy.stats import uniform

try:
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import StratifiedKFold
except ImportError:
    from sklearn.cross_validation import GridSearchCV
    from sklearn.cross_validation import RandomizedSearchCV
    from sklearn.cross_validation import StratifiedKFold
    

In [12]:
#определим random state для всех методов
RND_ST = 99

### Решение задачи:

#### А. Использовать методы

1) kNN
2) дерево принятия решения
3) логистическая регрессия


#### Б. Выбрать наилучшее решение


#### Предобработка

In [13]:
# предобработка, убираем категориальные признаки
def preproc_categorical(df_Data): 
    df_preproc_kNN = df_Data.copy()

    samlpe_col = []
    categorical_cols = [c for c in df_preproc_kNN.columns if (df_preproc_kNN[c].dtype.name == 'object' and c not in samlpe_col ) ]

    df_output_kNN = pd.get_dummies(df_preproc_kNN, columns=categorical_cols)
    
    return df_output_kNN

In [241]:
df_X_preproc = df_X.pipe(preproc_categorical)
df_X_preproc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24712 entries, 0 to 24711
Data columns (total 63 columns):
age                              24712 non-null int64
duration                         24712 non-null int64
campaign                         24712 non-null int64
pdays                            24712 non-null int64
previous                         24712 non-null int64
emp.var.rate                     24712 non-null float64
cons.price.idx                   24712 non-null float64
cons.conf.idx                    24712 non-null float64
euribor3m                        24712 non-null float64
nr.employed                      24712 non-null float64
job_admin.                       24712 non-null uint8
job_blue-collar                  24712 non-null uint8
job_entrepreneur                 24712 non-null uint8
job_housemaid                    24712 non-null uint8
job_management                   24712 non-null uint8
job_retired                      24712 non-null uint8
job_self-employed

In [15]:
df_X_test_preproc = df_X_test.pipe(preproc_categorical)

#### А.1.1 метод kNN

In [16]:
def solve_kNN(dataX, dataY, rand_state=123):
    
    solv_model = KNeighborsClassifier(random_state=rand_state)
    
    # гиперпараметры модели
    solv_param_grid = {
        'n_neighbors': randint(10, 25)
        ,'weights': ['distance', 'uniform']
    #    ,'algorithm': ['ball_tree', 'kd_tree', 'brute']
        ,'leaf_size': randint(25, 35)
    }

    cv = StratifiedKFold(n_splits=5, random_state=rand_state, shuffle=True)

    random_search = RandomizedSearchCV(solv_model, param_distributions=solv_param_grid, n_iter=100, n_jobs=-1,
                                       cv=cv, scoring='roc_auc', random_state=rand_state)
    random_search.fit(dataX, dataY)

    solv_bs = random_search.best_score_
    solv_bp = random_search.best_params_
    
    model = random_search.best_estimator_
    try:
        importances = model.feature_importances_
    except:
        importances = []
    
    result = {'model': model, 'importances': importances, 'best_score': solv_bs, 'best_params': solv_bp}
    
    return result

In [17]:
# подбор гиперпараметров для kNN, ответ = {'model','importances','best_score','best_params'}
result_kNN = solve_kNN(df_X_preproc, y, RND_ST)

In [18]:
result_kNN


{'model': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=1, n_neighbors=24, p=2,
            weights='distance'),
 'importances': [],
 'best_score': 0.9220535547658417,
 'best_params': {'leaf_size': 30, 'n_neighbors': 24, 'weights': 'distance'}}

#### А.1.2 Проверка на тестовой выборке

In [20]:
predict_kNN = result_kNN['model'].predict(df_X_test_preproc) 

In [23]:
predict_kNN

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

#### А.2 Дерево принятия решений

In [242]:
def solve_dTree(dataX, dataY, solv_param_grid, rand_state=123):

    solv_model = DecisionTreeClassifier(random_state=rand_state)
    
    # гиперпараметры модели
    solv_param_grid = {
        #'criterion': ['gini', 'entropy']
        'criterion': ['entropy']
        , 'max_depth': randint(4, 20)
        , 'min_samples_leaf': randint(15, 40)
        , 'class_weight': ['balanced']        
        #, 'class_weight': [None, 'balanced']        
    }

    cv = StratifiedKFold(n_splits=10, random_state=rand_state, shuffle=True)

    random_search = RandomizedSearchCV(solv_model, param_distributions=solv_param_grid, n_iter=100, n_jobs=-1,
                                       cv=cv, scoring='roc_auc', random_state=rand_state )
    random_search.fit(dataX, dataY)

    solv_bs = random_search.best_score_
    solv_bp = random_search.best_params_
    
    model = random_search.best_estimator_
    try:
        importances = model.feature_importances_
    except:
        importances = []
    
    result = {'model': model, 'importances': importances, 'best_score': solv_bs, 'best_params': solv_bp}
    
    return result

In [243]:
# подбор гиперпараметров для DecisionTreeClassifier, ответ = {'model','importances','best_score','best_params'}
result_dTree = solve_dTree(df_X_preproc, y, RND_ST)

In [244]:
result_dTree

{'model': DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
             max_depth=7, max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=35, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=123,
             splitter='best'),
 'importances': array([4.00127474e-03, 5.02481991e-01, 2.69407082e-03, 8.62383001e-03,
        1.62255158e-03, 6.31316921e-02, 4.00695254e-03, 8.08985057e-02,
        4.72374066e-02, 2.63789221e-01, 1.15944432e-03, 1.65553175e-03,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.91176814e-04, 0.00000000e+00, 2.06829096e-03, 0.00000000e+00,
        0.00000000e+00, 8.24185244e-04, 0.00000000e+00, 0.00000000e+00,
        1.08495019e-03, 0.00000000e+0

In [245]:
df_X_preproc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24712 entries, 0 to 24711
Data columns (total 63 columns):
age                              24712 non-null int64
duration                         24712 non-null int64
campaign                         24712 non-null int64
pdays                            24712 non-null int64
previous                         24712 non-null int64
emp.var.rate                     24712 non-null float64
cons.price.idx                   24712 non-null float64
cons.conf.idx                    24712 non-null float64
euribor3m                        24712 non-null float64
nr.employed                      24712 non-null float64
job_admin.                       24712 non-null uint8
job_blue-collar                  24712 non-null uint8
job_entrepreneur                 24712 non-null uint8
job_housemaid                    24712 non-null uint8
job_management                   24712 non-null uint8
job_retired                      24712 non-null uint8
job_self-employed

#### А.2.2 Проверка на тестовой выборке

In [156]:
predict_dTree = result_dTree['model'].predict(df_X_test_preproc) 

In [157]:
predict_dTree

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

#### А.3.1 Линейная регрессия

In [144]:
def solve_lReg(dataX, dataY, rand_state=123):

    solv_model = LogisticRegression(random_state=rand_state)
    
    # гиперпараметры модели
    solv_param_grid = {
        #'solver' : {'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'}
        #, 
        'max_iter' : randint(50, 150)
        , 'class_weight': [None, 'balanced']        
    }

    cv = StratifiedKFold(n_splits=5, random_state=rand_state, shuffle=True)

    random_search = RandomizedSearchCV(solv_model, param_distributions=solv_param_grid, n_iter=20, n_jobs=-1,
                                       cv=cv, scoring='roc_auc', random_state=rand_state)
    random_search.fit(dataX, dataY)

    solv_bs = random_search.best_score_
    solv_bp = random_search.best_params_
    
    model = random_search.best_estimator_
    try:
        importances = model.feature_importances_
    except:
        importances = []
    
    result = {'model': model, 'importances': importances, 'best_score': solv_bs, 'best_params': solv_bp}
    
    return result

In [145]:
# подбор гиперпараметров для DecisionTreeClassifier, ответ = {'model','importances','best_score','best_params'}
result_lReg = solve_lReg(df_X_preproc, y, RND_ST)

In [146]:
result_lReg

{'model': LogisticRegression(C=1.0, class_weight='balanced', dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=85,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=99,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
 'importances': [],
 'best_score': 0.9357035899929993,
 'best_params': {'class_weight': 'balanced', 'max_iter': 85}}

#### А.3.2 Проверка на тестовой выборке

In [148]:
predict_lReg = result_lReg['model'].predict(df_X_test_preproc) 

In [149]:
predict_lReg

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

### Лучшая модель

In [246]:
#result_kNN, result_dTree, result_lReg
print(result_kNN['best_score'])
print(result_dTree['best_score'])
print(result_lReg['best_score'])

0.9220535547658417
0.9379207141103798
0.9357035899929993


#### Попробуем упростить таблицу данных, убрав малозначимые характеристики 

In [254]:
index_col_serie = df_X_preproc.columns.to_series()

In [255]:
result_importances_serie = pd.Series(index=index_col_serie, data=result_dTree['importances']).sort_values()
#result_importances_serie

In [256]:
# собираем список колонок для исключения из учебной выборки
drop_list = []
for i in df_X_preproc.columns:
    if result_importances_serie[i] == 0 :
        drop_list.append(i)

In [258]:
df_X_preproc_new = df_X_preproc.drop(drop_list, axis=1)

##### Заново обучим модель

In [260]:
# подбор гиперпараметров для DecisionTreeClassifier, ответ = {'model','importances','best_score','best_params'}
result_dTree_new = solve_dTree(df_X_preproc_new, y, RND_ST)
result_dTree_new

{'model': DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
             max_depth=7, max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=35, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=123,
             splitter='best'),
 'importances': array([4.00127474e-03, 5.02481991e-01, 2.69407082e-03, 8.62383001e-03,
        1.27655891e-03, 6.29708875e-02, 4.16775716e-03, 8.08985057e-02,
        4.72374066e-02, 2.63789221e-01, 1.15944432e-03, 1.65553175e-03,
        1.91176814e-04, 2.06829096e-03, 8.24185244e-04, 1.08495019e-03,
        3.93929142e-03, 2.22401242e-03, 2.94407632e-03, 2.03865137e-03,
        1.95791975e-04, 9.62046199e-04, 2.23408462e-03, 3.36962753e-04]),
 'best_score': 0.9379081934790964,
 'best_params': {'class_weight': 'balanced',
  'criterion': 'entropy',
  'max_depth': 7,
  'min_samples_leaf': 35}}

In [262]:
# уберем "лишние" характеристики из тестовой выборки
df_X_test_preproc_new = df_X_test_preproc.drop(drop_list, axis=1)

In [264]:
predict_dTree_new = result_dTree_new['model'].predict(df_X_test_preproc_new) 
predict_dTree_new

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [265]:
s_id = df_test1['_id']
s_target_test_new = pd.Series(predict_dTree_new, index=s_id.values)
pd.DataFrame(s_target_test_new, columns=['target']).to_csv('data/target_submission_04_dtc.csv')

### Предсказание лучшей моделью запишем submission-файл

In [162]:
# серия с _id
s_id = df_test1['_id']
# серия с решениями модели DecisionTreeClassifier
s_target_test = pd.Series(predict_dTree, index=s_id.values)

In [159]:
df_submission = pd.DataFrame(s_target_test, columns=['target'])

In [160]:
df_submission.head()

,target
66810d8e6bf2b41c880a7bc6c8a1e295,0
ccac3879652b08cb8b44c1920fd93afa,0
fcccab4d7a76f70647f015f2c84c2af8,0
ed8399278c30678dab739045fa12b440,0
1d4d62ac5cabcb48bac7112813f290cb,0


In [161]:
df_submission.to_csv('data/target_submission_03_dtc.csv')

#### Сравним с логистической регрессией

In [150]:
# серия с _id
s_id = df_test1['_id']
s_target_test = pd.Series(predict_lReg, index=s_id.values)
df_submission = pd.DataFrame(s_target_test, columns=['target'])
df_submission.to_csv('data/target_submission_LR.csv')

## Дополнительные тесты

In [133]:
test_2_result_dTree = solve_dTree(df_X_preproc, y, RND_ST)

In [134]:
test_2_result_dTree

{'model': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=99,
             splitter='best'),
 'importances': array([0.001709  , 0.49918741, 0.00126963, 0.03841651, 0.        ,
        0.00166215, 0.0054787 , 0.03133907, 0.04284149, 0.35652204,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.00328634,
        0.  